<a href="https://colab.research.google.com/github/RShaya/ML_collab_draft1/blob/main/ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!unzip '/content/ML_Project.zip'

Archive:  /content/ML_Project.zip
   creating: ML_project/.idea/
  inflating: ML_project/.idea/misc.xml  
  inflating: ML_project/.idea/ML_project.iml  
  inflating: ML_project/.idea/modules.xml  
  inflating: ML_project/.idea/workspace.xml  
  inflating: ML_project/dev.out      
  inflating: ML_project/dev.p2.out   
 extracting: ML_project/dev.p3.out   
  inflating: ML_project/dev.prediction  
   creating: ML_project/EN/
  inflating: ML_project/EN/.DS_Store  
  inflating: ML_project/EN/dev.in    
  inflating: ML_project/EN/dev.out   
  inflating: ML_project/EN/train     
  inflating: ML_project/evalResult.py  
  inflating: ML_project/Instructions.txt  
  inflating: ML_project/ML.py        
  inflating: ML_project/what.py      


In [11]:
import numpy as np 

In [26]:
k=1
def words_hash(words):
    #hash all the words to integers, word is key and integer is value 
    hash_dict={'UNK':0}
    i=1
    for word in words:
        if word not in hash_dict:
            hash_dict[word]=i
            i+=1
    print("HashDict:")
    print(hash_dict)
    return hash_dict



In [49]:
doc = '/content/ML_project/EN/train'
def load_data(doc):
  O_tag = 0
  B_neutral = 0
  B_positive = 0
  B_negative = 0
  I_neutral = 0
  I_positive = 0
  I_negative = 0
  text= open(doc)
  words = []
  possible_tags =['O', 'B-neutral', 'B-positive', 'B-negative', 'I-neutral', 'I-positive', 'I-negative']
  tags=[]
  temp=[]
  array=text.read().split('\n')
  for element in array:
      sub = element.split(' ')
      # print(sub)
      s=len(sub)
      if sub[-1]=='' or sub[-1]==' ':
        pass
      else:
        word=sub[0]
        if s>2:
          for t in range (1,s):
            word+= ' '+ sub[t]
        tag= sub[-1] #the last string is definitley a tag 
        words.append(word)
        tags.append(tag)
        if tag=='O':
            O_tag+=1
        elif tag == 'B-neutral':
            B_neutral+=1
        
        elif tag == 'B-positive':
            B_positive+=1

        elif tag == 'B-negative':
            B_negative+=1
        
        elif tag == 'I-neutral':
            I_neutral+=1
        
        elif tag == 'I-positive':
            I_positive+=1
        
        elif tag == 'I-negative':
            I_negative+=1
  print("Words:")
  print(words)
  print("Tags:")
  print(tags)
  hash_dict= words_hash(words)
  tagcount= [O_tag, B_neutral, B_positive, B_negative, I_neutral, I_positive, I_negative]
  print(tagcount)
  return hash_dict, words, tags, tagcount

In [52]:
hash_dict, words, tags, tagcount = load_data(doc)

hashtag={'O':0, 
         'B-neutral' : 1 ,
         'B-positive' : 2,
         'B-negative' : 3,
         'I-neutral' : 4 ,
         'I-positive': 5,
         'I-negative': 6}
# print(hash_dict)
T= 7 # number of tags 


Words:
['【', '原价', '12', '.', '9', '元', '，', '劵', '后', '7', '.', '9', '元', '】', '女孩', '拼装', '积木', '公主', '城堡', '别墅', '系列', '兼容', '乐高', '儿童', '益智', '城市', '玩具', '6', '-', '10', '岁', '—', '—', '—', '复制', '这', '条', '信息', '，', '打开', '【', '手机', '淘宝', '】', '即可', '领券', '并', '下单', '￥', 'AANsIOGJ', '￥', 'http://t.cn/Ryh0PQz', '#', '阴阳师', '手游', '#', '急急如律令', '！', '感谢', '晴明', 'SAMA', '帮', '我', '召唤', '出', '了', '狸猫', '！', '养一养', '，', 'R', '的', '品级', '，', 'SSR', '的', '力量', '！', '٩', '͡', '[', '๏', '̯', '͡', '๏', ']', '۶', 'http://t.cn/RcXXIsB', '【', '云', '龙湾', '】', '山', '那边', '是', '海', '，', '海', '那边', '是', '山', '。', '山是', '铜鼓', '岭', '，', '海是', '云', '龙湾', '。', '山与海', '相依相偎', '，', '相互', '映衬', '，', '构成', '了', '奇特', '壮美', '的', '自然景观', '与', '人间', '胜迹', '。', '岭南', '，', '一碧万顷', '，', '海天', '无际', '，', '涛声', '不', '绝', '，', '浪花', '腾雪', '，', '那', '湛蓝', '澄碧', '的', '海水', '，', '让', '人', '一', '看', '就', '想', '跳下去', '洗个', '痛快', '，', '这', '就是', '云', '龙湾', '。', '【', '“', '招女婿', '”', '为何', '叫', '“', '入赘', '”', '】', '带', '

In [ ]:
def emmision_probability(words, tagword_dict):
    #Implementing emmision matrix as a series of dictionaries where each dict contains emmision probability for a each word in the set
    O_UNK_count= 1+(1/(O_tag+1))
    B_neutral_count=1+( 1/(B_neutral+1))
    B_positive_UNK_count= 1+(1/(B_positive+1))
    B_negative_count= 1+(1/(B_negative+1))
    I_neutral_UNK_count= 1+(1/(I_neutral+1))
    I_positive_UNK_count= 1+(1/(I_positive+1))
    I_negative_UNK_count= 1+(1/(I_negative+1))

    O_tag_emmision_dict={'UNK': O_tag_UNK_count} #records the emmision probability for each words for O tags 
    B_neutral_emmision_dict={'UNK': B_neutral_UNK_count} # records emmision probability for each word for B-neutral tags 
    B_positive_emmision_dict={'UNK': B_positive_UNK_count} # records emmision probability for each word for B-positive tags 
    B_negative_emmision_dict={'UNK': B_negative_UNK_count} # records emmision probability for each word for B-negative tags 
    I_neutral_emmision_dict={'UNK': I_neutral_UNK_count} # records emmision probability for each word for I-neutral tags 
    I_positive_emmision_dict={'UNK': I_positive_UNK_count} # records emmision probability for each word for I-positive tags 
    I_negative_emmision_dict={'UNK': I_negative_UNK_count} # records emmision probability for each word for I-negative tags 

    for word in words:
        try:
            O_count = tagword_dict[(word, 'O')]
        except:
            O_count=0
        try:
            B_neutral_count= tagword_dict[(word, 'B-neutral')]
        except:
            B_neutral_count=0
        try:
            B_positive_count = tagword_dict[(word, 'B-positive')]
        except:
            B_positive_count=0
        try:
            B_negative_count = tagword_dict[(word, 'B-negative')]
        except:
            B_negative_count=0
        try:
            I_neutral_count = tagword_dict[(word, 'I-neutral')]
        except:
            I_neutral_count=0
        try:
            I_positive_count = tagword_dict[(word, 'I-positive')]
        except:
            I_positive_count=0
        try:
            I_negative_count = tagword_dict[(word, 'I-negative')]
        except:
            I_negative_count=0

        if word not in O_tag_emmision_dict:
            O_tag_emmision_dict[word]= (O_count/(O_tag+1))+1 # add one so there is no 0 probability. Helps during sentiment analysis 
        
        if word not in B_neutral_emmision_dict:
            B_neutral_emmision_dict[word]= (B_neutral_count/(B_neutral_tag+1))+1
        
        if word not in B_positive_emmision_dict:
            B_positive_emmision_dict[word]= (B_positive_count/(B_positive+1))+1
        
        if word not in B_negative_emmision_dict:
            B_negative_emmision_dict[word]= (B_negative_count/(B_negative+1))+1
        
        if word not in I_neutral_emmision_dict:
            I_neutral_emmision_dict[word]= (I_neutral_count/(I_neutral+1))+1
        
        if word not in n_emmision_dict:
            I_positive_emmision_dict[word]= (I_positive_count/(I_positive+1))+1
        
        if word not in I_negative_emmision_dict:
            I_negative_emmision_dict[word]= (I_negative_count/(I_negative+1))+1
# print(B_neutral_emmision_dict)
# print(B_positive_emmision_dict)
# print(B_negative_emmision_dict)
# print(I_neutral_emmision_dict)
# print(I_positive_emmision_dict)
# print(I_negative_emmision_dict)

O_tag_emmision_dict, B_neutral_emmision_dict, B_positive_emmision_dict, B_negative_emmision_dict, I_neutral_emmision_dict, I_positive_emmision_dict,I_negative_emmision_dict= emmision_probability(words, tagword_dict)

NameError: ignored

In [59]:
def emission_calc(hash_dict, words, tags, hashtag, tagcount):
    n= len(hash_dict)
    M= np.zeros((T, n)) # rows are tgs, colomns are words
    for i in range(T):
        M[i][hash_dict['UNK']]= k
    for i in range(len(words)):
        # if words[i]== 'happy':
        #     print('happy')
        tag= tags[i]
        wordhash= hash_dict[words[i]]
        taghash=hashtag[tag]
        # print( (taghash, wordhash))
        M [taghash][wordhash]+=1
    t=0
    for row in M:
            count = tagcount[t]
            row[:]= [(f/(count+k)) for f in row]
            # print(tagcount[t])
            t+=1
            # if np.sum(row) != 1:
            #   print(False)
            #   print(np.sum(row))
    
    return M
emission_matrix= emission_calc(hash_dict, words, tags, hashtag, tagcount)
print(emission_matrix)


[[1.27040589e-05 4.61157340e-03 3.81121768e-05 ... 1.27040589e-05
  1.27040589e-05 0.00000000e+00]
 [3.08737265e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.11688312e-04 8.11688312e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.85062714e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.23065799e-04 7.23065799e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.35294118e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [60]:
def emmision(u, o, emission_matrix):
    if u=='START' or u== 'STOP':
        return 0
    word= hash_dict[o]
    tag= hashtag[u]
    return float(emission_matrix[tag][word])



In [ ]:
# needs to be done 
def analyse_sentiment(p_emmision_dict, n_emmision_dict, test_file):
    text= open(test_file)
    output_seq=[]
    test_array=text.read().split('\n')
    for word in test_array:
        if word in p_emmision_dict:
            ratio= p_emmision_dict[word]/n_emmision_dict[word]
        else:
            ratio= p_emmision_dict['UNK']/n_emmision_dict['UNK']
        if ratio>=1:
            output_seq.append('P')
        else:
            output_seq.append('N')
    return output_seq


output=analyse_sentiment(p_emmision_dict, n_emmision_dict, 'test_file.txt')
print(output)

In [67]:
def transition_calc(tags, hashtag):
  # tag_dict={'START': 0, 'P': 1, 'N': 2, 'STOP':3}
  n= len(hashtag) # number of possible tag 
  # T: sequence of numbers corresponding to sequence of tags from input 
  T= []
  for tag in tags:
      T.append(hashtag[tag])
  # print(T)

  #create a 4x4 matrix (dimensions can be changed based on number of tags) that will record transition probabilities 

  M = np.array([[0]*n for i in range(n)],dtype=float)
  transition_matrix = np.array([[0]*n for i in range(n)],dtype=float)

  #count number of time u is followed by v 

  for (i,j) in zip(T,T[1:]):
      M[i][j] += 1 
  for z in range(n):
      transition_matrix[z] = M[z]/sum(M[z])

  transition_matrix= np.nan_to_num(transition_matrix) # for u = STOP all values are zero 
  return transition_matrix

transition_matrix= transition_calc(tags, hashtag)
print(transition_matrix)
# for row in transition_matrix:
#   if np.sum(row)!= 1:
#     print(False)

[[0.9420172  0.03951063 0.0150801  0.00339207 0.         0.
  0.        ]
 [0.47992588 0.01822112 0.001853   0.         0.5        0.
  0.        ]
 [0.39642567 0.00243704 0.01299756 0.         0.         0.58813972
  0.        ]
 [0.65942029 0.00362319 0.         0.02536232 0.         0.
  0.3115942 ]
 [0.44225834 0.01653835 0.00285144 0.         0.53835187 0.
  0.        ]
 [0.51013025 0.00506512 0.00868307 0.         0.         0.47612156
  0.        ]
 [0.62222222 0.         0.         0.01481481 0.         0.
  0.36296296]]


In [ ]:
def transition_probability(u,v,transition_matrix):
    if u == 'STOP' or v == 'START':
        return 0
    else:
        return float(transition_matrix[tag_dict[u]][tag_dict[v]])

In [ ]:
viterbi_vals={}
layer_maxprob={}

def normal_viterbi(input, k, last_tag, tags):
    
    # Uncomment this code when tags include START and STOP
#input is the sequence length, the last tag, all possible tags (nodes in every layer), observation sequence return max probability path length and path sequence as an array
    if k==1: # k is the number of tags in the tag seq (including start and stop) so when one tag is left it must be START
        if last_tag=='START':
            return 1
        else:
            return 0
    
    #Base case for tags without start and stop
    # if k==1: # k is 1 means only one element in the list hence base case  
    #     locmax=0
    #     for u in tags:
    #         # print(input[1])
    #         p=emmision(u, input[0],emission_matrix)
    #         if p>locmax:
    #             locmax=p
    #             best_tag=u
    #     # print('locmax ' + str(locmax))
    #     viterbi_vals[(k,last_tag)]= best_tag
    #     return locmax 

        
    max_prob= 0
    
    for u in tags:
        # print ("k: "+ str(k))
        # print(u)
        # print(input)
        # print(last_tag)
        if last_tag== 'STOP':
            prob= normal_viterbi(input,k-1,u, tags) * transition_probability(u, last_tag, transition_matrix)
        else:
            if k==2:
                # print(input)
                # print("Base: ")
                # print(normal_viterbi(input, k-1,u, tags))
                prob= normal_viterbi(input, k-1,u, tags) * transition_probability(u, last_tag, transition_matrix) * emmision(last_tag, input[0], emission_matrix)
                # if u=='START': 
                #     print("START")
                #     print(str(normal_viterbi(input, k-1,u, tags)) +" "+ str(transition_probability(u, last_tag, transition_matrix)) + " " + str( emmision(last_tag, input[0], emission_matrix)))
                
            else:
                # print(k)
                # print(input[k-2])
                prob= normal_viterbi(input, k-1,u, tags) * transition_probability(u, last_tag, transition_matrix) * emmision(last_tag, input[k-2], emission_matrix) # since last tag is stop, we need to see k-1th word. In list, the count starts from 0, so k-1th word has index k-2
            # print('output: ' + str(prob))
        if prob > max_prob:
            # print('here')
            # print(max_prob)
            besttag=u
            max_prob= prob
        layer_maxprob[k]= max_prob
        try:
            viterbi_vals[(k,last_tag)]= besttag
        except:
            viterbi_vals[(k,last_tag)]= 'NIL'
            # best_seq[k]=u #check , kept k for dummy dataset 
    # print(best_seq)
    # best_seq = best_seq.reverse 
    return max_prob

In [ ]:
def backtracking(k, last_tag):
    tag_seq= [last_tag]
    while k>1: #CHECK: initially coded k>=1, but gives 9 tags for 8 inputs, k>1 gives 8 tags, correct output 
        next= viterbi_vals[((k,last_tag))]
        tag_seq.append(next)
        # print(k)
        k=k-1
        last_tag=next
        
    return tag_seq

In [ ]:
def top_five(max_prob, best_seq, tags, k, input, last_tag):
    top_five={tuple(best_seq): max_prob}
#consider the k-1 layer to find the second max route from the best node in the k-2 layer
    prev_node= best_seq[k-3]
    print(prev_node)
    # best_so_far= viterbi_vals[(k-2, prev_node)]
    best_so_far= normal_viterbi(input, k-2, prev_node,tags)#can use memoized result to avoid running viterbi again
    print(best_so_far)
    for u in tags: #
        if u != best_seq[k-2]: #check index 
            # prob= best_so_far * transition_probability(prev_node, u) * emmision(u, input[k]) * transition_probability(u,last_tag) # for actual dataset 
            print(input[k-1])
            prob= best_so_far * transition_probability(prev_node, u, transition_matrix) * emmision(u, input[k-2], emission_matrix) * transition_probability(u,last_tag, transition_matrix)* emmision(last_tag, input[k-1], emission_matrix) #dummy dataset 
            seq= best_seq[:k-2]+ [u] + [last_tag]
            if len(top_five) < 5:
                flag=False
                for value in top_five.values():
                    if prob == value: #do not add duplicate values 
                        flag=True
                if flag==False:
                    top_five[tuple(seq)]= prob
                
            if len(top_five)==5:
                sorted_top_five= sorted(top_five.values())
                top_five=insert(prob, seq, sorted_top_five)
    return top_five


def insert(prob, seq, sorted_top_five):
    if sorted_top_five[dict.keys()[-1]]> prob:
        return sorted_top_five
    else:
        last_key= dict.keys()[-1]
        del sorted_top_five[last_key]
        sorted_top_five[seq]=prob #sorting is done in main function 
    return sorted_top_five